In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import operator
from infomap import infomap
from pymongo import MongoClient
%matplotlib inline

In [2]:
filename = "/Users/arefindk/Desktop/github_merged_pull/merged_pull_multi_noselfloop_digraph_timestamp_lang.graphml"
G = nx.read_graphml(filename)

In [3]:
print "Nodes ", G.number_of_nodes()
print "Edges ", G.number_of_edges()

Nodes  505028
Edges  3184242


In [4]:
## Load mongodb
client = MongoClient("mongodb://localhost:27017/")
db = client.github_follow
users = db.users

In [5]:
## Some lists
removedNodes = list()
allNodes = G.nodes()
numNodes = len(allNodes)
count = 0
## Getting language attribute of the user
for currentNode in allNodes:
    ## Finding a user with specific login name from the network
    findDoc = {"login":currentNode}
    result = users.find_one(findDoc)

    if not result:
        removedNodes.append(currentNode)
    else:
        nativeLanguage = result['native_language']

        if nativeLanguage== None:
            removedNodes.append(currentNode)
        else:
            G.node[currentNode]['lang'] = str(nativeLanguage)
            G.node[currentNode]['public_repos'] = int(result['public_repos'])
            f = "%d/%m/%Y %H:%M:%S"
            G.node[currentNode]['created_at'] = result['created_at'].strftime(f)
#             if not result['company'] == None:
#                 G.node[currentNode]['company'] = str(result['company'])
#             else:
#                 G.node[currentNode]['company'] = ""
#             if not result['location'] == None:
#                 G.node[currentNode]['location'] = str(result['location'])
#             else:
#                 G.node[currentNode]['location'] = ""
            G.node[currentNode]['type'] = str(result['type'])
            G.node[currentNode]['latest_followers'] = int(result['followers'])
            G.node[currentNode]['latest_followings'] = int(result['following'])
    count += 1
    if count % 10000 == 0:
        print "Remaining ", numNodes - count

Remaining  495028
Remaining  485028
Remaining  475028
Remaining  465028
Remaining  455028
Remaining  445028
Remaining  435028
Remaining  425028
Remaining  415028
Remaining  405028
Remaining  395028
Remaining  385028
Remaining  375028
Remaining  365028
Remaining  355028
Remaining  345028
Remaining  335028
Remaining  325028
Remaining  315028
Remaining  305028
Remaining  295028
Remaining  285028
Remaining  275028
Remaining  265028
Remaining  255028
Remaining  245028
Remaining  235028
Remaining  225028
Remaining  215028
Remaining  205028
Remaining  195028
Remaining  185028
Remaining  175028
Remaining  165028
Remaining  155028
Remaining  145028
Remaining  135028
Remaining  125028
Remaining  115028
Remaining  105028
Remaining  95028
Remaining  85028
Remaining  75028
Remaining  65028
Remaining  55028
Remaining  45028
Remaining  35028
Remaining  25028
Remaining  15028
Remaining  5028


In [6]:
print len(removedNodes)

57928


In [7]:
all_nodes_set = set(allNodes)

In [9]:
removed_nodes_set = set(removedNodes)

In [10]:
trimmed_node_set = all_nodes_set - removed_nodes_set

In [11]:
len(trimmed_node_set)

447100

In [12]:
trimmed_subgraph = G.subgraph(trimmed_node_set)

In [16]:
print trimmed_subgraph.number_of_nodes()
print trimmed_subgraph.number_of_edges()

447100
2817073


In [18]:
langs = nx.get_node_attributes(trimmed_subgraph,'lang')
lang_set = set(langs.values())
print len(lang_set), lang_set

239 set(['Mercury', 'Mako', 'TypeScript', 'DCPU-16 ASM', 'PureBasic', 'Objective-C++', 'DTrace', 'Delphi', 'Lean', 'NewLisp', 'Jupyter Notebook', 'Rebol', 'Frege', 'GAP', 'AspectJ', 'Brightscript', 'Shell', 'Web Ontology Language', 'AppleScript', 'Elm', 'Nix', 'SuperCollider', 'Nginx', 'Opa', 'Elixir', 'SAS', 'Agda', 'Logos', 'SMT', 'D', 'ooc', 'Kotlin', 'Standard ML', 'Objective-C', 'Batchfile', 'Bison', 'Oz', 'Mirah', 'Ox', 'Objective-J', 'NetLinx', 'LoomScript', 'Handlebars', 'Gosu', 'Gettext Catalog', 'Swift', 'Ceylon', 'Smarty', 'Pure Data', 'Arc', 'Brainfuck', 'IGOR Pro', 'APL', 'Hack', 'Go', 'Visual Basic', 'PHP', 'Cirru', 'SQF', 'Glyph', 'Groovy', 'Java', 'Scala', 'OpenSCAD', 'ApacheConf', 'Makefile', 'BitBake', 'Perl', 'HCL', 'Lua', 'LLVM', 'GDScript', 'Verilog', 'Stata', 'Factor', 'Haxe', 'Component Pascal', 'Forth', 'Red', 'COBOL', 'Groff', 'XProc', 'LSL', 'eC', 'CoffeeScript', 'HTML', 'Awk', 'UrWeb', 'Lex', 'API Blueprint', 'Idris', 'Ruby', 'LilyPond', 'PLpgSQL', 'Modula-2'

In [19]:
user_type = nx.get_node_attributes(trimmed_subgraph,'type')

In [20]:
same_type_of_users = {}
for k,v in user_type.iteritems():
    same_type_of_users.setdefault(v,[]).append(k)

In [21]:
print same_type_of_users.keys()

['Organization', 'User']


In [26]:
users_organization = same_type_of_users['Organization']
print len(users_organization)

33314


In [27]:
users_user = same_type_of_users['User']
print len(users_user)

413786


In [28]:
trimmed_subgraph_users = G.subgraph(users_user)
trimmed_subgraph_organizations = G.subgraph(users_organization)

In [29]:
print trimmed_subgraph_users.number_of_edges()

2134332


In [30]:
print trimmed_subgraph_organizations.number_of_edges()

1032


In [41]:
components_G = nx.strongly_connected_components(trimmed_subgraph_users)
components_G = sorted(components_G, key = lambda c:len(c), reverse = True)
largest_strongly_connected_component = components_G[0]
induced_strongly_connected_subgraph = trimmed_subgraph_users.subgraph(largest_strongly_connected_component)
print induced_strongly_connected_subgraph.number_of_edges()
## Now converting it from a Multi_Di_Graph to a simple Di_Graph
induced_strongly_connected_subgraph = nx.DiGraph(induced_strongly_connected_subgraph)
print induced_strongly_connected_subgraph.number_of_edges()

941128
271305


In [42]:
induced_strongly_connected_subgraph_int = nx.convert_node_labels_to_integers(induced_strongly_connected_subgraph, first_label = 0, ordering = 'default', label_attribute = "login")

In [44]:
print len(largest_strongly_connected_component)
print induced_strongly_connected_subgraph_int.number_of_nodes(), induced_strongly_connected_subgraph_int.number_of_edges()

55457
55457 271305


In [45]:
nx.write_graphml(induced_strongly_connected_subgraph_int, "trimmed_merged_pull_users_strong_int.graphml")